In [1]:
import sys
print(sys.version)
print("executable: ")
print(sys.executable)
import tensorflow as tf
print(tf.__version__)
import tensorflow_datasets as tfds
print(tfds.__version__)
#print("TFDS version:", tfds.__version__)
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

3.8.10 (default, Nov 22 2023, 10:22:35) 
[GCC 9.4.0]
executable: 
/usr/bin/python


2025-01-29 14:54:24.262643: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-29 14:54:24.375019: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-29 14:54:25.131810: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-01-29 14:54:25.131891

2.11.0
4.9.2


In [2]:
atn1 = tf.keras.models.load_model('ATN_base_model_2NDHALF.keras')
atn2 = tf.keras.models.load_model('ATN_simpleCNN.keras')
atn3 = tf.keras.models.load_model('ATN_StrongCNN.keras')
atn1.trainable = False  # Freeze weights
atn2.trainable = False  # Freeze weights
atn3.trainable = False  # Freeze weights
atns = [atn1, atn2, atn3]

2025-01-29 14:54:31.102220: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12978 MB memory:  -> device: 0, name: NVIDIA A16, pci bus id: 0000:1c:00.0, compute capability: 8.6


In [3]:
#loading mnist data from tensorflow
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
#mnist = tfds.load(name='mnist', split='train', as_supervised=True)
# Returns both train and test split separately
mnist_train, mnist_test = tfds.load('mnist', split=['train', 'test'], as_supervised=True)
def preprocess_image(image, label):
    image = image
    #image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1] # i believe this is not in the original code
    label = tf.one_hot(label, depth=10)  # One-hot encode the label
    return image, label
mnist_train = mnist_train.map(preprocess_image)
mnist_test = mnist_test.map(preprocess_image)
num_train_samples = len(list(mnist_train))
mnist_train = mnist_train.take(num_train_samples // 2) #only first half of mnist train
print(len(mnist_train))

30000


In [4]:
#defining the network
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) *n_conv2

In [5]:
#initialising the weights with random values
# NOT USED IN THIS CODE WITH KERAS IMPLEMENTATION
weights = {
    "wc1" : tf.Variable(tf.random.normal([conv1_k, conv1_k, input_channels, n_conv1])),
    "wc2" : tf.Variable(tf.random.normal([conv2_k, conv2_k, n_conv1, n_conv2])),
    "wh1" : tf.Variable(tf.random.normal([input_size_to_hidden, n_hidden])),
    "wo" : tf.Variable(tf.random.normal([n_hidden, n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random.normal([n_conv1])),
    "bc2" : tf.Variable(tf.random.normal([n_conv2])),
    "bh1" : tf.Variable(tf.random.normal([n_hidden])),
    "bo" : tf.Variable(tf.random.normal([n_out])),
}

In [6]:
#functions for layers
def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x, weights, padding="SAME", strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding = "SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1])

In [7]:
#function for forward prop
#This function is edited now with keras functionality. weights are now handled internally but can still be accessed with get_weights method
def cnn(x):
    # Define the layers using Keras API
    #x = layers.Reshape((input_height, input_width, input_channels))(x)
    
    conv1 = layers.Conv2D(n_conv1, kernel_size=(conv1_k, conv1_k), strides=stride_conv1, padding="same", activation="relu")(x)
    conv1_pool = layers.MaxPooling2D(pool_size=(max_pool1_k, max_pool1_k))(conv1)

    conv2 = layers.Conv2D(n_conv2, kernel_size=(conv2_k, conv2_k), strides=stride_conv2, padding="same", activation="relu")(conv1_pool)
    conv2_pool = layers.MaxPooling2D(pool_size=(max_pool2_k, max_pool2_k))(conv2)

    # Flatten the output from convolutional layers
    flatten = layers.Flatten()(conv2_pool)

    # Fully connected layer
    hidden = layers.Dense(n_hidden, activation="relu")(flatten)

    # Dropout
    dropout = layers.Dropout(0.2)(hidden)

    # Output layer (logits)
    output = layers.Dense(n_out, activation = 'softmax')(dropout)
    
    return output
    
    #x = tf.reshape(x, shape = [-1 ,input_height, input_width, input_channels])
    #conv1 = conv(x, weights['wc1'], biases['bc1'], stride_conv1)
    #conv1_pool = maxpooling(conv1, max_pool1_k)
    
    #conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    #conv2_pool = maxpooling(conv2, max_pool2_k)
    
    #hidden_input = tf.reshape(conv2_pool, shape = [-1, input_size_to_hidden])
    #hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1'])
    #hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    #hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob) 
   
    #output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo']) #no softmax activation function since the loss function handles it already
    #return output

In [8]:
#x = tf.keras.Input(shape=(input_pixels,))
x = tf.keras.Input(shape=(input_height, input_width, input_channels))
y = tf.keras.Input(shape=(n_out,))
#pred is the model
pred = cnn(x)

In [9]:
model = models.Model(inputs=[x], outputs=pred)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) #changed from 0.01 to 0.001
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
train_data = mnist_train.batch(32)
test_data = mnist_test.batch(32)
# Get a single batch from the test data (a batch is a tuple of (image, label))
#single_batch = next(iter(test_data))

# single_batch is a tuple, where single_batch[0] contains the image and single_batch[1] contains the label
#image, label = single_batch

# You can print or inspect the first image and label from the batch
#print(image[0].numpy())  # The first image in the batch
#print(label[0].numpy())  # The label for the first image


In [10]:
def visualize_samples(original, adversarial, index=None):
    """
    Visualize original and adversarial samples side-by-side.
    
    Args:
        original (np.array): The original sample (e.g., shape (28, 28, 1)).
        adversarial (np.array): The adversarial sample (same shape as original).
        index (int, optional): The index of the sample (useful for debugging).
    """
    original = np.squeeze(original)  # Remove extra dimensions if present
    adversarial = np.squeeze(adversarial)  # Remove extra dimensions if present
    
    # Set up the plot
    fig, axs = plt.subplots(1, 2, figsize=(8, 4))
    fig.suptitle(f"Original vs. Adversarial (Index: {index})", fontsize=16)
    
    # Plot original
    axs[0].imshow(original, cmap='gray')
    axs[0].set_title("Original")
    axs[0].axis('off')
    
    # Plot adversarial
    axs[1].imshow(adversarial, cmap='gray')
    axs[1].set_title("Adversarial")
    axs[1].axis('off')
    
    plt.show()

In [11]:
def create_partly_adv_data(batch_images, fraction=0.5):
    batch_size = batch_images.shape[0]
    num_adversarial = int(fraction * batch_size)
    batch_images = np.array(batch_images)
    original_batch_images = batch_images.copy()
    
    # Select random indices within the batch
    indices = np.random.choice(batch_size, num_adversarial, replace=False)
    
    # Choose random ATN and generate adversarial samples
    chosen_atn = np.random.choice(atns)
    adversarial_examples = chosen_atn.predict(batch_images[indices], verbose=0)

    # Create a new batch with a mix of original and adversarial examples
    batch_images[indices] = adversarial_examples
    
    #for index in indices:
    #    visualize_samples(original_batch_images[index], batch_images[index])
    
    # Convert back to TensorFlow tensor
    batch_images = tf.convert_to_tensor(batch_images, dtype=tf.float32)
    
    return batch_images

In [12]:
def cnn_train_step(batch_images, batch_labels):
    mixed_images = create_partly_adv_data(batch_images, fraction=0.5)
    loss, accuracy = model.train_on_batch(mixed_images, batch_labels)
    return loss, accuracy  # Return both loss and accuracy

In [13]:
import sys
from time import time

num_epochs = 160  # Adjust as needed
for epoch in range(num_epochs):
    cnn_start_time = time()
    print(f"Epoch {epoch + 1}/{num_epochs}")
    
    train_data_iterator = iter(train_data)  # Create an iterator
    total_loss = 0
    total_accuracy = 0
    num_batches = len(train_data)  # Total number of batches

    for batch_number in range(num_batches):  # Loop over batches
        try:
            # Get the next batch
            x, labels = next(train_data_iterator)
            loss, accuracy = cnn_train_step(x, labels)  

            # Accumulate metrics
            total_loss += loss
            total_accuracy += accuracy

            # Progress bar display
            progress = int((batch_number + 1) / num_batches * 30)  # 30-character progress bar
            sys.stdout.write(
                f"\r[{'=' * progress}{'.' * (30 - progress)}] "
                f"{batch_number + 1}/{num_batches} - loss: {loss:.4f} - accuracy: {accuracy:.4f}"
            )
            sys.stdout.flush()

        except StopIteration:
            break  # In case the iterator runs out of data

    # End of epoch metrics
    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches
    elapsed_time = time() - cnn_start_time
    print(f"\nEpoch {epoch + 1} completed in {elapsed_time:.2f}s - loss: {avg_loss:.4f} - accuracy: {avg_accuracy:.4f}")

Epoch 1/160


2025-01-29 14:54:49.410250: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8400
2025-01-29 14:54:57.009395: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


[==============================] 938/938 - loss: 0.4732 - accuracy: 0.8750
Epoch 1 completed in 149.19s - loss: 0.7014 - accuracy: 0.9142
Epoch 2/160
[==============================] 938/938 - loss: 0.9193 - accuracy: 0.8125
Epoch 2 completed in 134.04s - loss: 0.1193 - accuracy: 0.9651
Epoch 3/160
[==============================] 938/938 - loss: 1.0247 - accuracy: 0.6875
Epoch 3 completed in 137.30s - loss: 0.1031 - accuracy: 0.9685
Epoch 4/160
[==============================] 938/938 - loss: 0.5659 - accuracy: 0.8750
Epoch 4 completed in 133.22s - loss: 0.0917 - accuracy: 0.9726
Epoch 5/160
[==============================] 938/938 - loss: 0.2739 - accuracy: 0.9375
Epoch 5 completed in 133.11s - loss: 0.0827 - accuracy: 0.9754
Epoch 6/160
[==============================] 938/938 - loss: 0.3496 - accuracy: 0.9375
Epoch 6 completed in 132.54s - loss: 0.0759 - accuracy: 0.9781
Epoch 7/160
[==============================] 938/938 - loss: 0.0027 - accuracy: 1.0000
Epoch 7 completed in 135.

KeyboardInterrupt: 

In [14]:
test_loss, test_acc = model.evaluate(test_data) #dropout is automatically set to 1.0 when calling model.evaluate
print(f'Test accuracy: {test_acc}')
print("summary: ")
model.summary()

313/313 [==============================] - 2s 4ms/step - loss: 1.0646 - accuracy: 0.9850
Test accuracy: 0.9850000143051147
summary: 
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                        

In [15]:
model.save('EAT_100e_001lr.keras')